## Simultaneously adjusting role and language level

In [1]:
import sys
sys.path.append("/home/mihaelabaksic/proj/2023-languagelearning")
sys.path.append("/home/mihaelabaksic/proj/2023-languagelearning/src") 
sys.path.append("/home/mihaelabaksic/proj/2023-languagelearning/src/templates") 

%load_ext autoreload
%autoreload 2

In [11]:
from single_run_thread import SingleRunThread
from prompt_builder import PromptBuilder
from utils import load_template, load_cefr
from parameters import GPTParameters
import keys

import openai

openai.api_key = keys.OPENAI_API_KEY

In [3]:
builder = PromptBuilder()
t = SingleRunThread()

In [4]:
examples = [
    {
        'setting' : 'In a cafe',
        'roles': ['waiter','customer'],
        'GPT_role': 'waiter',
        'user_role': 'customer'        
    },
    {
        'setting': 'Enchanted forest',
        'roles': ['explorer', 'forest spirit'],
        'GPT_role': 'forest spirit',
        'user_role': 'explorer'
    },
    {
        'setting' : 'Talking to Yann LeCun on a conference afterparty',
        "roles": ["conference attendee", "Yann LeCun"],
        "GPT_role": "Yann LeCun",
        "user_role": "conference attendee",
    }
]

In [53]:
template_alternatives = [
    # arguments: GPT_role, user_role, language_level, cefr_description
    "Rephrase the sentence with 2 goals: 1. The sentence should use tone and voice of {0} speaking to {1}, 2. The sentence should only use {2} level of language proficiency, defined as {3}: ",
    
    # CoT, first level, then role
    # arguments: GPT_role, user_role, language_level, cefr_description
    "Rephrase the sentence with 2 goals: 1. The sentence should use tone and voice of {0} speaking to {1}, 2. The sentence should only use {2} level of language proficiency, defined as {3}.\nExamples:\n\nSentence:I can't give you that drink because we don't have it.\nTask:The sentence should only use C2 level of language.\nThought: Is this C2 level of language? If not, rephrase it to be C2 level.\nTask: The sentence should use tone and voice of a waiter talking to a customer.\nThought: Does it fit the role of a waiter? If not, rephrase it to fit the role.\nRefinement:I am very sorry, but we do not serve that drink, would you fancy a cocktail instead? Our bartenders make exquisit cocktails.\n\nSentence:Mother, I wish to go home.\nTask:The sentence should only use A2 level of language proficiency\nThought:Is the sentence on A2 level of language? If not, rephrase it to be A2.\nTask:The sentence should use tone and voice of a child speaking to its mother.\nThought: Does it match the role of a small child? If not rephrase it to match the role.\nRefinement:Mommy, I wanna go homee!.\n\n Return ONLY refinement.\n\nSentence: ",

    # Cot, first role, then level
    # arguments: language_leve, cefr_description, GPT_role, user_role
    "Rephrase the sentence with 2 goals: 1. The sentence should only use {2} level of language proficiency, defined as {3}.  2. The sentence should use tone and voice of {0} speaking to {1}.\nExamples:\n\nSentence:I can't give you that drink because we don't have it.\nTask: The sentence should use tone and voice of a waiter talking to a customer.\nThought: Does it fit the role of a waiter? If not, rephrase it to fit the role.\nTask:The sentence should only use C2 level of language.\nIs this C2 level of language? If not, rephrase it to be C2 level.\nRefinement:I am very sorry, but we do not serve that drink, would you fancy a cocktail instead? Our bartenders make exquisit cocktails.\n\nSentence:Mother, I wish to go home.\nTask:The sentence should use tone and voice of a child speaking to its mother.\nThought: Does it match the role of a small child? If not rephrase it to match the role.\nTask:The sentence should only use A2 level of language proficiency.\nThought: Is the sentence on A2 level of language? If not, rephrase it to be A2.\nRefinement:Mommy, I wanna go homee!.\n\n Return ONLY refinement.\n\nSentence:",

]

### Need to adjust role

In [54]:
lang_level = 'B1'

b1_description = load_cefr('B1', '../../templates/language_levels_cefr.json')

sentences = [
    'We have a great variety of coffees.',
    'Ye man I haven\'t seen you around these trees before.',
    'Hello, mate, how ya doing\'.'   
]

#### Instruction only

In [37]:
for sentence, example in zip(sentences, examples):
    builder.reset()
    
    builder.add_template(template_alternatives[0], (example['GPT_role'], example['user_role'], lang_level, b1_description) )
    builder.add_template(sentence)
    
    prompt = builder.build()
    
    print(t.send(prompt))

We offer a wide selection of coffee options for you to choose from.
Greetings, dear explorer! It appears that your presence amidst these majestic trees is a novelty to my forest spirit eyes.
Hello there, my friend, how are you doing?


#### Instruction + CoT (first lnguage level, then role)

In [55]:
for sentence, example in zip(sentences, examples):
    builder.reset()
    
    builder.add_template(template_alternatives[1], (example['GPT_role'], example['user_role'], lang_level, b1_description) )
    builder.add_template(sentence)
    
    prompt = builder.build()
    
    print(t.send(prompt))

We have a wide selection of coffees.
Greetings, traveler! I haven't come across your presence amidst these majestic trees before.
Hey there, buddy, how are ya?


#### Instruction + CoT (first role, then language level)

In [56]:
for sentence, example in zip(sentences, examples):
    builder.reset()
    
    builder.add_template(template_alternatives[2], (lang_level, b1_description, example['GPT_role'], example['user_role']) )
    builder.add_template(sentence)
    
    prompt = builder.build()
    
    print(t.send(prompt))

We have a wide selection of coffees available.
Refinement: Hey there, I haven't come across you in this forest before.
Hey there, buddy, how's it going?


### Need to adjust language level

In [57]:
lang_level = 'A1'
a2_description = load_cefr('A1', '../../templates/language_levels_cefr.json')

sentences = [
     'I will be right back with your order, would you fancy something else.',
     'We have never had a human here before, this is such an exsquisit opportunity.',
     'I would like to visit you in your lab some day, perhaps to discuss our furhter cooperation.'
]

#### Instruction only

In [20]:
for sentence, example in zip(sentences, examples):
    builder.reset()
    
    builder.add_template(template_alternatives[0], (example['GPT_role'], example['user_role'], lang_level, a2_description) )
    builder.add_template(sentence)
    
    prompt = builder.build()
    
    print(t.send(prompt))

I'll be back soon with your food. Do you want anything else?
Oh, dear explorer, never before has a human graced our presence in this enchanted forest. What a wondrous and extraordinary chance this is for us.
I want to come to your lab one day, maybe to talk about how we can work together more.
